# <font color=Red> Aim of the Project
### In this project, we will classify malaria cell images as either infected or uninfected.  
A deep learning model for image classification will be developed and deployed using Hugging Face.
https://huggingface.co/spaces/HarunDemircioglu11/Malaria_Cell_Detection

<img src='https://img.freepik.com/fotos-premium/mosquitos-aedes-aedes-transmitem-doencas-dia-mundial-da-malaria-25-de-abril_10221-19519.jpg?w=996'>

In [25]:
import cv2 #OpenCV kut. goruntuleri okuma yazma duzenleme ve analiz etme
import pandas as pd #
import os

In [27]:
from pathlib import Path


current_path = Path.cwd()

# Çalışma dizinini yazdır
print(f"Mevcut çalışma dizini: {current_path}")

Mevcut çalışma dizini: C:\Users\harunn\Music\ARTIFICIAL INTELLIGENCE\MALERIA


- This line defines the two possible classes for your image classification task: 'Parasitized' (cells infected with parasites) and 'Uninfected' (healthy cells).

- This line specifies the directory path where the cell images used for training and testing are stored.


In [29]:
labels=['Parasitized','Uninfected']
img_path='cell_images/'

- An empty list called `img_list` is created to store the file paths of all images that will be read from the folders.
- Another empty list called `label_list` is created to store the corresponding label (class) for each image.
- The outer loop iterates through each class label (e.g., 'Parasitized' and 'Uninfected').
- For each label, the inner loop visits the respective folder and iterates through all image files in that folder.
- The full file path of each image is added to `img_list`.
- The corresponding class label is added to `label_list`, ensuring that each image is matched with its correct label.


In [ ]:
img_list=[] # ici bos listem resimleri okuyacagim sonr okudugum resimleri listemin icine atacagim
label_list=[] # okudugum resmin karsisina etiketimi koyacagim 
for label in labels: # kansere ya da kanser olmyanlarin klasorune git dedim once klasordeki her bir resmi ziyaret edip her birinin ismini alacagim
    for img_file in os.listdir(img_path+label): #labeli alinca onun altindaki kanser ve non kansere gittim
        img_list.append(img_path+label+'/'+img_file)
        label_list.append(label)

- The code first imports the required libraries: `Path` from `pathlib` for easier path operations and `os` for directory scanning.
- Two class labels, 'Parasitized' and 'Uninfected', are defined, and the main image directory is set using `Path`.
- Two empty lists, `img_list` and `label_list`, are created to store the image file paths and their corresponding labels.
- For each class label, the code constructs the path to the relevant folder (e.g., `cell_images/Parasitized`).
- It checks if the folder exists. If not, it prints a warning message.
- If the folder exists, it uses `os.scandir()` to efficiently scan all files within the folder.
- For each file:
    - The loop is limited to the first 500 images per class to avoid loading too many files.
    - It checks if the file is a regular image file with the extension .jpg, .jpeg, or .png.
    - Files named "thumbs.db" are skipped, as they are not image data.
    - The file path is appended to `img_list`, and its label is appended to `label_list`.
- At the end, the code prints the total number of loaded images.
- The comment indicates that this code was originally provided by Çağla Derin Sahin.


In [32]:
from pathlib import Path
import os

labels = ['Parasitized', 'Uninfected']
img_path = Path('cell_images')

img_list = []
label_list = []

for label in labels:
    folder_path = img_path / label
    if folder_path.exists():
        with os.scandir(folder_path) as files:
            for i, file in enumerate(files):
                if i >= 500:  # 
                    break
                if file.is_file() and file.name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    if "thumbs.db" in file.name.lower():
                        continue
                    img_list.append(str(file.path))
                    label_list.append(label)
    else:
        print(f"{folder_path} bulunamadı!")

print(f"{len(img_list)} görüntü yüklendi.")
## Bu kod Cagla inandan alinmistir


1000 görüntü yüklendi.


- This line creates a DataFrame named `df` with two columns:  
  - `'img'`: contains the file paths of all images,  
  - `'label'`: contains the corresponding class label (e.g., 'Parasitized' or 'Uninfected') for each image.  
- This DataFrame organizes the image data and labels in a structured table, making it easier to use for further data processing or model training.


In [34]:
os.listdir('cell_images')

['Parasitized', 'Uninfected']

In [36]:
df=pd.DataFrame({'img':img_list,'label':label_list})

In [38]:
df.head()

,img,label
0,cell_images\Parasitized\C100P61ThinF_IMG_20150...,Parasitized
1,cell_images\Parasitized\C100P61ThinF_IMG_20150...,Parasitized
2,cell_images\Parasitized\C100P61ThinF_IMG_20150...,Parasitized
3,cell_images\Parasitized\C100P61ThinF_IMG_20150...,Parasitized
4,cell_images\Parasitized\C100P61ThinF_IMG_20150...,Parasitized


- This line creates a dictionary named `d` that maps each class label to a numerical value:
  - `'Parasitized'` is mapped to `1` (representing infected),
  - `'Uninfected'` is mapped to `0` (representing healthy).
- This mapping is typically used to convert categorical class labels into numerical labels for machine learning models.


In [40]:
import matplotlib.pyplot as plt

In [42]:
d={'Parasitized':1, 'Uninfected':0}

In [44]:
df['encode_label']=df['label'].map(d)

In [46]:
df.tail()

,img,label,encode_label
995,cell_images\Uninfected\C107P68ThinF_IMG_201509...,Uninfected,0
996,cell_images\Uninfected\C107P68ThinF_IMG_201509...,Uninfected,0
997,cell_images\Uninfected\C107P68ThinF_IMG_201509...,Uninfected,0
998,cell_images\Uninfected\C107P68ThinF_IMG_201509...,Uninfected,0
999,cell_images\Uninfected\C107P68ThinF_IMG_201509...,Uninfected,0


In [47]:
import numpy as np

- An empty list `x` is created to store the processed image data.
- For each image file path in the `'img'` column of the DataFrame:
  - The image is read from disk using OpenCV (`cv2.imread`).
  - The image data type is set to `'uint8'` to ensure consistent format.
  - The image is resized to 170x170 pixels for uniformity.
  - The pixel values are normalized to the range [0, 1] by dividing by 255.0.
  - The processed image is converted to a NumPy array.
  - The resulting array is appended to the list `x`.
- At the end of the loop, `x` contains all images as normalized, resized NumPy arrays, ready for model training.

In [50]:
x = []
for img_path in df['img']:
    img = cv2.imread(img_path)
    img = img.astype('uint8')
    img = cv2.resize(img, (170, 170))
    img = img / 255.0 
    img = np.array(img)
    x.append(img)

In [51]:
x=np.array(x)

In [52]:
y=df['encode_label']

- A sequential deep learning model is defined for image classification.
- The input layer expects images with shape (170, 170, 3) (height, width, color channels).
- The first convolutional layer has 32 filters, a 3x3 kernel size, and uses the ReLU activation function.
- A max pooling layer with a 2x2 window reduces the spatial dimensions.
- The second convolutional layer has 64 filters, again with a 3x3 kernel and ReLU activation.
- Another max pooling layer is applied.
- The output from the convolutional layers is flattened into a one-dimensional vector.
- A dense (fully connected) layer with 128 units is added.
- The final dense layer has 2 output units with softmax activation, suitable for binary classification (infected vs. uninfected).
- The model is compiled with the Adam optimizer, sparse categorical crossentropy loss (for integer labels), and accuracy as the evaluation metric.


In [53]:
from sklearn.model_selection import train_test_split

In [55]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=20,random_state=42)

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Reshape, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [57]:
model=Sequential()
model.add(Input(shape=(170,170,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(2, activation='softmax')) # 10 fakli cevap classification 0-9 a kadar olan rakamlar
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [61]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=15, verbose=1)

Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.4756 - loss: 8.9380 - val_accuracy: 0.4500 - val_loss: 0.7015
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 37s 916ms/step - accuracy: 0.6045 - loss: 0.6840 - val_accuracy: 0.5500 - val_loss: 0.6914
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.6031 - loss: 0.6379 - val_accuracy: 0.5000 - val_loss: 0.6610
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 37s 931ms/step - accuracy: 0.7626 - loss: 0.5358 - val_accuracy: 0.5500 - val_loss: 0.7084
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 29s 945ms/step - accuracy: 0.8345 - loss: 0.4135 - val_accuracy: 0.4500 - val_loss: 0.8038
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 39s 878ms/step - accuracy: 0.8995 - loss: 0.2834 - val_accuracy: 0.5000 - val_loss: 0.7923
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9205 - loss: 0.2210 - val_accuracy: 0.4000 - val_loss: 1.2813
Epoch 8/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 31s 977ms/step - accuracy: 0.9799 - loss: 0.1239 - val_accuracy: 0.5

In [63]:
model.save('myn_cnn_model.h5')

## Detailed Project Summary

- **Data Preparation:**  
  The dataset was organized into two folders, one for each class label. All image file paths and their corresponding labels were collected and stored in a DataFrame for easier manipulation and processing.

- **Label Encoding:**  
  The class labels were mapped to numerical values (1 for 'Parasitized', 0 for 'Uninfected') to be compatible with machine learning algorithms.

- **Image Preprocessing:**  
  All images were resized to 170x170 pixels and normalized to have pixel values between 0 and 1. This standardization helped the model learn more effectively and reduced computational complexity.

- **Model Architecture:**  
  A convolutional neural network (CNN) was designed using the Keras Sequential API. The model included two convolutional layers with ReLU activation, max pooling layers to reduce dimensionality, a fully connected dense layer, and a final softmax layer for binary classification.

- **Training and Evaluation:**  
  The model was trained using the Adam optimizer and sparse categorical crossentropy loss. After training, the model achieved a **validation accuracy of 98%**, indicating excellent performance in distinguishing infected and uninfected cell images.

- **Deployment:**  
  The trained model is suitable for deployment on platforms such as Hugging Face, allowing for real-time malaria detection from microscopic images.

**Conclusion:**  
This project demonstrates that deep learning can be highly effective for medical image classification tasks. Achieving 98% accuracy, the model provides a reliable tool for supporting malaria diagnosis and has the potential to improve healthcare workflows.


https://huggingface.co/spaces/HarunDemircioglu11/Malaria_Cell_Detection